This first example of the basic usage of the Pyriod package was demonstrated at the [TASC5/KASC12 workshop](https://web.mit.edu/tasc5/) in Boston 22-26 July 2019.  It shows how to reproduce the frequency analysis of a pulsating helium-atmosphere white dwarf presented in [Bell et al. (A&A, 632, A42)](https://ui.adsabs.harvard.edu/abs/2019A%26A...632A..42B).  

See the online [tutorial](https://pyriod.readthedocs.io/en/latest/tutorial/) for more info about how to use Pyriod.

In [1]:
%matplotlib widget
import lightkurve as lk
from Pyriod import Pyriod

In [2]:
lc = lk.search_lightcurve('TIC 900086743',mission='TESS',sector=22)[0].download().normalize().remove_outliers(7)

In [61]:

index=10
lc["flux_err"][index] = 90
lc.flux_err[index]

<Quantity 90.>

In [42]:
np.mean(lc.flux_err),np.std(lc.flux)

(<Quantity 0.57366705>, <Quantity 0.4144364>)

In [35]:
perunweighted = lc.to_periodogram(dy=1)

In [36]:
perweighted =  lc.to_periodogram(dy=lc.flux_err)

In [37]:
plt.plot(perunweighted.frequency,perunweighted.power)
plt.plot(perweighted.frequency,perweighted.power,alpha=0.5)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [3]:
import numpy as np
time = np.arange(0,27,1/(24*30))
freq = 100
freq2 = 123.3243232234
amp = 0.01
amp2 = 0.05
noise = 0.01 #sqrt(flux)
flux = 1 + amp*np.sin(2*np.pi*freq*time) + noise*np.random.randn(len(time)) #+ amp2*np.sin(2*np.pi*freq2*time+.1213)
lc = lk.LightCurve(time=time,flux=flux,flux_err=noise)
lc.scatter()
import matplotlib.pyplot as plt
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
#Download light curve of a DBV white dwarf pulsator observed by TESS
#Then smooth and remove outliers
#lc = lk.search_lightcurve('TIC 257459955',mission='TESS',sector=3)[0].download().flatten(2161).remove_outliers(5)

In [4]:
#Instantiate Pyriod object
pyriod = Pyriod(lc,freq_unit='day')

/Users/keatonb/anaconda3/lib/python3.7/site-packages/astropy/units/quantity.py:479: RuntimeWarning: invalid value encountered in sqrt
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)


In [5]:
#Interact with time series
pyriod.TimeSeries()

In [6]:
#Interact with periodogram
pyriod.Periodogram()

In [7]:
#Interact with frequency solution
pyriod.Signals()

In [9]:
np.sum(pyriod.lc['resid']**2/(noise**2.))/len(time-6)

1.0216276099170674

In [10]:
#amp err = 0.3727
#expected: 0.1014
# amp 2 = 0.1007
np.sqrt(2/len(time))*noise*1e3

0.10143010324169743

In [18]:
1.9686/(len(time)-6)

0.00010129669651126891

In [9]:
from IPython.core.display import display, HTML
fr = HTML(pyriod.fit_result._repr_html_())
type(fr)


IPython.core.display.HTML

In [10]:
fr

In [69]:
f = open("table.css", "r")
css = f.read()
print(css)
f.close()

<style  type="text/css" >
    table {
      border: none;
      border-collapse: collapse;
      border-spacing: 0;
      color: black;
      font-size: 12px;
      table-layout: fixed;
    }
    thead {
      border-bottom: 1px solid black;
      vertical-align: bottom;
    }
    tr, th, td {
      text-align: right;
      vertical-align: middle;
      padding: 0.5em 0.5em;
      line-height: normal;
      white-space: normal;
      max-width: none;
      border: none;
    }
    th {
      font-weight: bold;
    }
    tbody tr:nth-child(odd) {
      background: #f5f5f5;
    }
    tbody tr:hover {
      background: rgba(66, 165, 245, 0.2);
    }
</style>


In [72]:
HTML(css+pyriod.fit_result._repr_html_())

In [11]:
display(fr)

In [ ]:
HTML

In [21]:
accordion = widgets.Accordion(children=[widgets.HTML(pyriod.fit_result._repr_html_())], titles=('Fit report'))
accordion

Accordion(children=(HTML(value="<h2> Model</h2> Model(sin, prefix='f0') <h2>Fit Statistics</h2><table><tr><td>…

In [12]:
import ipywidgets as widgets
widgets.HTML(
    value=pyriod.fit_result._repr_html_()
)

HTML(value="<h2> Model</h2> Model(sin, prefix='f0') <h2>Fit Statistics</h2><table><tr><td>fitting method</td><…

In [9]:
#Interact with log
pyriod.Log()

In [14]:
pyriod.fitvalues

,include,freq,fixfreq,freqerr,amp,fixamp,amperr,phase,fixphase,phaseerr,per,pererr
f0,True,10.599042,False,0.000368,0.227711,False,0.0043,0.192886,False,0.003005,8151.679801,0.283132
f1,True,21.200157,False,0.002527,0.033224,False,0.0043,0.185071,False,0.020623,4075.441602,0.485811


In [15]:
#Show all panels in tabs (may be buggy in Python 3)
pyriod.Pyriod()

In [21]:
1/(2*pyriod.nyquist)

0.0027777402738138335

In [95]:
nyqphase = ((lc.time.value + .32334212) % (0.5/pyriod.nyquist))/(0.5/pyriod.nyquist)
nyqphase

array([0.79325097, 0.79327243, 0.79329422, ..., 0.79081713, 0.79079232,
       0.79076718])

In [96]:
(2*(2/60/24))

0.002777777777777778

In [97]:
pyriod.nyquist

360.00486057935194

In [98]:
plt.figure()
plt.hist(nyqphase)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [101]:
lc.time.value

array([1900.442374  , 1900.4437629 , 1900.4451518 , ..., 1926.49479665,
       1926.49618548, 1926.49757432])

In [102]:
import numpy as np
y = np.sin(2*np.pi*nyqphase)
x = np.cos(2*np.pi*nyqphase)

In [105]:
import matplotlib.pyplot as plt
plt.scatter(x,y,s=2)
plt.scatter(np.mean(x),np.mean(y))
plt.xlim(-1.1,1.1)
plt.ylim(-1.1,1.1)
plt.show()

nan


/Users/keatonb/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in double_scalars
  after removing the cwd from sys.path.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [115]:
nyqqual = np.sqrt(np.mean(x)**2.+np.mean(y)**2.)
nyqqual

0.9842709516605033